In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

In [ ]:
start = '2013-01-01'
end = '2023-12-21'
stock = 'GOOG'

data = yf.download(stock, start, end)

In [ ]:
data.reset_index(inplace=True)

In [ ]:
data

In [ ]:
ma_100_days = data.Close.rolling(100).mean()

In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(data['Date'], ma_100_days, 'r', label='100-day Moving Average')
plt.plot(data['Date'], data['Close'], 'g', label='Closing Price')
plt.title('100-day Moving Average and Closing Price of Google Stock')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
ma_200_days = data.Close.rolling(200).mean()

In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(data['Date'], ma_100_days, 'r', label='100-day Moving Average')
plt.plot(data['Date'], ma_200_days, 'b', label='200-day Moving Average')
plt.plot(data['Date'], data['Close'], 'g', label='Closing Price')
plt.title('100-day and 200-day Moving Averages and Closing Price of Google Stock')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
data.dropna(inplace=True)

In [ ]:
data_train = pd.DataFrame(data.Close[0: int(len(data)*0.80)])
data_test = pd.DataFrame(data.Close[int(len(data)*0.80): len(data)])

In [ ]:
data_train.shape[0]

In [ ]:
data_test.shape[0]

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))

In [ ]:
data_train_scale = scaler.fit_transform(data_train)

In [ ]:
x = []
y = []

for i in range(100, data_train_scale.shape[0]):
    x.append(data_train_scale[i-100:i])
    y.append(data_train_scale[i,0])
    

In [ ]:
x, y = np.array(x), np.array(y)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense, Dropout, LSTM
from keras.models import Sequential

In [ ]:
model = Sequential()
model.add(LSTM(units = 50, activation = 'relu', return_sequences = True,
    input_shape = ((x.shape[1],1))))
model.add(Dropout(0.2))

model.add(LSTM(units = 60, activation='relu', return_sequences = True))
model.add(Dropout(0.3))

model.add(LSTM(units = 80, activation = 'relu', return_sequences = True))
model.add(Dropout(0.4))

model.add(LSTM(units = 120, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(units =1))

In [ ]:
model.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [ ]:
model.fit(x,y, epochs = 50, batch_size =32, verbose =1)

In [ ]:
model.summary()

In [ ]:
pas_100_days = data_train.tail(100)

In [ ]:
data_test = pd.concat([pas_100_days, data_test], ignore_index=True)

In [ ]:
data_test_scale  =  scaler.fit_transform(data_test)

In [ ]:
x = []
y = []

for i in range(100, data_test_scale.shape[0]):
    x.append(data_test_scale[i-100:i])
    y.append(data_test_scale[i,0])
x, y = np.array(x), np.array(y)

In [ ]:
y_predict = model.predict(x)

In [ ]:
scale =1/scaler.scale_

In [ ]:
y_predict = y_predict*scale

In [ ]:
y = y*scale

In [ ]:
plt.figure(figsize=(14, 8))
plt.plot(y , color='g', linestyle='-', marker='o', markersize=2, label='Original Price')
plt.plot(y_predict, color='r', linestyle='--', marker='x', markersize=2, label='Predicted Price')
plt.title('Original and Predicted Stock Prices')
plt.xlabel('Time')
plt.ylabel('Price (USD)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
x_train_lr = np.array([data_train_scale[i - 100:i].flatten() 
                    for i in range(100, data_train_scale.shape[0])])
y_train_lr = np.array([data_train_scale[i, 0] 
                    for i in range(100, data_train_scale.shape[0])])


In [ ]:
x_test_lr = np.array([data_test_scale[i - 100:i].flatten() 
                    for i in range(100, data_test_scale.shape[0])])
y_test_lr = np.array([data_test_scale[i, 0] 
                    for i in range(100, data_test_scale.shape[0])])

In [ ]:
lr_model = LinearRegression()
lr_model.fit(x_train_lr, y_train_lr)

In [ ]:
y_predict_lr = lr_model.predict(x_test_lr)

In [ ]:
x_test_lr = []
y_test_lr = []

for i in range(100, len(data_test)):
    x_test_lr.append(data_test.values[i - 100: i, 0])
    y_test_lr.append(data_test.values[i, 0])
    
_test_lr, y_test_lr = np.array(x_test_lr), np.array(y_test_lr)

In [ ]:
scale = 1 / scaler.scale_[0]
y_predict_lr = y_predict_lr * scale
y_test_lr = y_test_lr * scale

In [ ]:
plt.figure(figsize=(14, 8))
plt.plot(y_test_lr, "g", label="Original Price", linewidth=2)
plt.plot(y_predict_lr, "r", label="Predicted Price (Linear Regression)", linestyle='--', linewidth=2)
plt.title('Stock Price Prediction using Linear Regression')
plt.xlabel("Time")
plt.ylabel("Price")
plt.grid(True)
plt.legend(loc="best")
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
mse = mean_squared_error(y_test_lr, y_predict_lr)
r2 = r2_score(y_test_lr, y_predict_lr)
print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
data['Target'] = np.where(data['Close'].shift(-1) > data['Close'], 1, 0)
data.dropna(inplace=True)

data_train_lr = data[0:int(len(data) * 0.80)]
data_test_lr = data[int(len(data) * 0.80):]

In [ ]:
x_train_lr = data_train_lr[['Open', 'High', 'Low', 'Close']]
y_train_lr = data_train_lr['Target']

x_test_lr = data_test_lr[['Open', 'High', 'Low', 'Close']]
y_test_lr = data_test_lr['Target']

In [ ]:
logistic_model = LogisticRegression()
logistic_model.fit(x_train_lr, y_train_lr)

In [ ]:
y_predict_lr = logistic_model.predict(x_test_lr)

In [ ]:
accuracy = accuracy_score(y_test_lr, y_predict_lr)
accuracy_percentage = accuracy * 100
print(f'Accuracy of Logistic Regression: {accuracy_percentage:.2f}%')

In [ ]:
model.save('Stock Predictions Model.keras')